<a href="https://colab.research.google.com/github/Imran123885/Harvard_AI_Bootcamp_Work/blob/main/imran_mirza_d2h1vc_mini_llm_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini-LLMs
Harvard AI Bootcamp

## Make a copy of this notebook! Editing directly will not be saved.

Let's build GPT with a Shakespeare dataset!

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

Hyperparameters are adjustable settings or configurations that determine a machine learning model's structure, learning rate, or optimization process. They are set before training and remain constant throughout the learning process. Properly tuning hyperparameters can significantly affect a model's performance, generalization ability, and convergence speed.

In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 50
eval_interval = 100
learning_rate = 1e-3
device = None
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

1. **batch_size:** It defines the number of independent sequences processed in parallel during each iteration of model training, influencing the efficiency and resource utilization of the training process.

2. **block_size:** This hyperparameter sets the maximum context length for predictions, determining the size of input sequences and the range of dependencies the model can capture.

3. **max_iters:** It specifies the maximum number of iterations (training steps) during model training, controlling the duration of the training process.

4. **eval_interval:** This hyperparameter sets how often the model's performance is evaluated on the training and validation sets, aiding in monitoring training progress and preventing overfitting.

5. **learning_rate:** It determines the step size in updating the model's parameters during optimization, influencing the convergence and stability of the training process.

6. **device:** It dynamically selects the computing device (GPU or CPU) available for training the model, adapting to the hardware environment.

7. **eval_iters:** It sets the number of iterations used for estimating the loss during evaluation, impacting the reliability of performance assessment.

8. **n_embd:** This hyperparameter defines the dimensionality of the model's embeddings, influencing the model's capacity to represent and learn complex patterns in the data.

9. **n_head:** It specifies the number of self-attention heads in the model, determining the diversity and parallel processing capability of attention mechanisms.

10. **n_layer:** This hyperparameter sets the number of transformer blocks or layers in the model, affecting its depth and capacity to capture hierarchical features.

11. **dropout:** It controls the probability of dropout regularization during training, where this hyperparameter influences model generalization and prevents overfitting.

In [ ]:
# TODO: change device to use GPU if GPU is available, otherwise use CPU
gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")
device = torch.device('cuda') if gpu_avail else torch.device('cpu')
print(device)

Is the GPU available? True
cuda


In [ ]:
# getting the Shakespear dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-12-20 07:47:12--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-12-20 07:47:12 (32.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
import pandas as pd

torch.manual_seed(1337)

# TODO: open the file, read the input data input.txt and store that in a variable called text
df = pd.read_table('input.txt')
text = df['First Citizen:']

['Before we proceed any further, hear me speak.',
 'All:',
 'Speak, speak.',
 'First Citizen:',
 'You are all resolved rather to die than to famish?',
 'All:',
 'Resolved. resolved.',
 'First Citizen:',
 'First, you know Caius Marcius is chief enemy to the people.',
 'All:',
 "We know't, we know't.",
 'First Citizen:',
 "Let us kill him, and we'll have corn at our own price.",
 "Is't a verdict?",
 'All:',
 "No more talking on't; let it be done: away, away!",
 'Second Citizen:',
 'One word, good citizens.',
 'First Citizen:',
 'We are accounted poor citizens, the patricians good.',
 'What authority surfeits on would relieve us: if they',
 'would yield us but the superfluity, while it were',
 'wholesome, we might guess they relieved us humanely;',
 'but they think we are too dear: the leanness that',
 'afflicts us, the object of our misery, is as an',
 'inventory to particularise their abundance; our',
 'sufferance is a gain to them Let us revenge this with',
 'our pikes, ere we become r

In [ ]:
# TODO: compute all the unique characters that occur in this text, store that in chars
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
text2 = ' '.join(texts for texts in text)
chars = word_tokenize(text2)
chars = [char.lower() for char in chars if char.isalpha()]
vocab_size = len(chars)
chars

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['before',
 'we',
 'proceed',
 'any',
 'further',
 'hear',
 'me',
 'speak',
 'all',
 'speak',
 'speak',
 'first',
 'citizen',
 'you',
 'are',
 'all',
 'resolved',
 'rather',
 'to',
 'die',
 'than',
 'to',
 'famish',
 'all',
 'resolved',
 'resolved',
 'first',
 'citizen',
 'first',
 'you',
 'know',
 'caius',
 'marcius',
 'is',
 'chief',
 'enemy',
 'to',
 'the',
 'people',
 'all',
 'we',
 'we',
 'first',
 'citizen',
 'let',
 'us',
 'kill',
 'him',
 'and',
 'we',
 'have',
 'corn',
 'at',
 'our',
 'own',
 'price',
 'a',
 'verdict',
 'all',
 'no',
 'more',
 'talking',
 'o',
 'let',
 'it',
 'be',
 'done',
 'away',
 'away',
 'second',
 'citizen',
 'one',
 'word',
 'good',
 'citizens',
 'first',
 'citizen',
 'we',
 'are',
 'accounted',
 'poor',
 'citizens',
 'the',
 'patricians',
 'good',
 'what',
 'authority',
 'surfeits',
 'on',
 'would',
 'relieve',
 'us',
 'if',
 'they',
 'would',
 'yield',
 'us',
 'but',
 'the',
 'superfluity',
 'while',
 'it',
 'were',
 'wholesome',
 'we',
 'might',
 'gu

In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val

KeyError: ignored

In [ ]:
# TODO: split in train_data and val_data based on n
train_data = None
val_data = None

The get_batch function appears to be responsible for generating a batch of input-target pairs from a given dataset

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

One-head self-attention refers to a specific instance where a single attention mechanism or "head" is employed. Self-attention mechanisms are a key component in transformer architectures, allowing the model to weigh different parts of the input sequence differently, based on learned attention scores.



In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

The purpose of using multiple attention heads is to allow the model to attend to different parts of the input sequence simultaneously, enabling the capture of diverse patterns and dependencies

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

FeedForward module is a component that consists of one or more linear layers followed by non-linear activation functions. The purpose of a feedforward module is to introduce non-linearity into the network and enable it to learn complex mappings from input to output.



In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

Bigram models are a type of statistical language model used for predicting the likelihood of a word based on the occurrence of its preceding word in a sequence. These models assume that the probability of a word depends only on the previous word, capturing local dependencies in the data. Bigram models are simple but effective, commonly used in tasks like text generation, machine translation, and information retrieval. However, they have limitations in handling long-range dependencies and contextual nuances compared to more advanced models like neural language models.

In [ ]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


In [ ]:
# TODO: create a PyTorch optimizer with AdamW and pass in the model parameters and learning rate
optimizer = None

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))